In [1]:
import os
import json
import shutil
import copy
from zipfile import ZipFile
from modules.utils import *
from modules.geometry import *

In [2]:
path = "/Users/samo/Documents/School/Bioinf/CVAT_polygon_interpolator/data/task_sonoscape_2021-09_2021-10-18_003_20211001_151030_31.avi_backup_2022_04_06_06_53_20 (1).zip"

In [3]:
# comment out unwanted
shapes_lst = [
    "lungslidingpresent",
    "lungslidingabsent",
    # "aline",
    "bline"
]

In [4]:
with ZipFile(path) as myzip:
    with myzip.open("annotations.json") as json_file:
        annots = json.load(json_file)


In [5]:
shapes = copy.deepcopy(annots[0]["shapes"])
filtered_shapes = [
    shape
    for shape in shapes
    if shape["type"] == "polygon"
    and shape["label"] in shapes_lst
]
indicies = get_indicies(shapes)

In [6]:
shapes_by_frame = group_shapes_by_frame(filtered_shapes)
remove_duplicates(shapes_by_frame)
assign_id(shapes_by_frame)
align_points(shapes_by_frame, annots[0]['shapes'])
interpolator(shapes_by_frame, indicies, annots)

TypeError: unhashable type: 'slice'

In [7]:
# create output folder if it doesn't exist
if not os.path.exists("output"):
    os.makedirs("output")

# copy the original file to output folder
shutil.copy(path, "output/" + path.split("/")[-1])

remove_from_zip("output/" + path.split("/")[-1], "annotations.json")
with zipfile.ZipFile("output/" + path.split("/")[-1], 'a') as newzip:
    newzip.writestr("annotations.json", json.dumps(annots))
